In [1]:
import os
import subprocess
import glob
import shutil

# Run test

In [45]:
command_template = "python run_infer.py \
--gpu='0' \
--nr_types={} \
--type_info_path=type_info.json \
--batch_size=8 \
--model_mode={} \
--model_path={} \
--nr_inference_workers=8 \
--nr_post_proc_workers=12 \
tile \
--input_dir=dataset/sample_tiles/imgs/ \
--output_dir=dataset/sample_tiles/pred/ \
--mem_usage=0.1 \
--draw_dot \
--save_qupath"

info = {
    "pannuke": ["6", "fast", "pretrained/hovernet_fast_pannuke_type_tf2pytorch.tar"],
    "monusac": ["5", "fast", "pretrained/hovernet_fast_monusac_type_tf2pytorch.tar"],
    "consep_type": ["5", "original", "pretrained/hovernet_original_consep_type_tf2pytorch.tar"],
    "consep_notype": ["0", "original", "pretrained/hovernet_original_consep_notype_tf2pytorch.tar"],
    "cpm17": ["0", "original", "pretrained/hovernet_original_cpm17_notype_tf2pytorch.tar"],
    "kumar": ["0", "original", "pretrained/hovernet_original_kumar_notype_tf2pytorch.tar"]
}

info = {
    # "kumar": ["0", "original", "pretrained/hovernet_original_kumar_notype_tf2pytorch.tar"],
    # "cutom": ["0", "original", "logs/00/net_epoch=10.tar"],
    "kumar2": ["0", "original", "pretrained/kumar_retrain.tar"]
}

In [46]:
output_dirs = [r"C:\Users\toom\cuhk-proj\hover_net\dataset\sample_tiles\pred\overlay" ,
                r"C:\Users\toom\cuhk-proj\hover_net\dataset\sample_tiles\pred\mat"]
sav_dir = r"C:\Users\toom\Desktop\test_kumar"   # Path to the new dir

for dataset, v in info.items():
    # Format string
    line = command_template.format(*v)

    # Run command and wait
    process = subprocess.Popen(['powershell.exe', 'conda activate hovernet; ' + line], \
                                stdout=subprocess.PIPE, \
                                cwd="C:/Users/toom/cuhk-proj/hover_net")
    process.wait()

    if process.returncode != 0:
        print(dataset)
        continue

    # Move files to other dir
    for output_dir in output_dirs:
        files = glob.glob(os.path.join(output_dir, "*"))
        curr_sav_dir = os.path.join(sav_dir, dataset, os.path.basename(output_dir))
        if not os.path.exists(curr_sav_dir):
            os.makedirs(curr_sav_dir)
        for i in files:
            shutil.move(i, os.path.join(curr_sav_dir, os.path.basename(i)))


# Run eval

In [1]:

import sys
sys.path.append('../')
import numpy as np

import torch
import torch.utils.data as data
from importlib import import_module
from run_utils.utils import convert_pytorch_checkpoint
from models.hovernet.utils import crop_to_shape, dice_loss, mse_loss, msge_loss, xentropy_loss


In [21]:
model_args =  {
                'nr_types'   : None,
                'mode'       : "original",
            }
        
model_path = r"C:\Users\toom\cuhk-proj\hover_net\pretrained\hovernet_original_kumar_notype_tf2pytorch.tar"

In [29]:
model_desc = import_module("models.hovernet.net_desc")
model_creator = getattr(model_desc, "create_model")

net = model_creator(**model_args)
saved_state_dict = torch.load(model_path)["desc"]
saved_state_dict = convert_pytorch_checkpoint(saved_state_dict)

net.load_state_dict(saved_state_dict, strict=True)
net = torch.nn.DataParallel(net)
net = net.to("cuda")

module_lib = import_module("models.hovernet.run_desc")
run_step = getattr(module_lib, "infer_step")
what = lambda input_batch: run_step(input_batch, net)

In [36]:
mat1 = np.load(r"C:\Users\toom\cuhk-proj\hover_net\dataset\training_data\kumar\kumar\train\540x540_164x164\TCGA-18-5592-01Z-00-DX1_000.npy")
mat1 = mat1.reshape((1, 540, 540, 4))[:, :, :, :3]

In [47]:
import scipy.io as sio

In [121]:
names = glob.glob(r"C:\Users\toom\Desktop\data\kumar\test_diff\Labels/*")
names = [os.path.basename(i)[:-4] for i in names]
gt_dir = r"C:\Users\toom\Desktop\data\kumar\test_diff\Labels"
k_dir = r"C:\Users\toom\Desktop\test_kumar\kumar\mat"
k2_dir = r"C:\Users\toom\Desktop\test_kumar\kumar2\mat"
c_dir = r"C:\Users\toom\Desktop\test_kumar\custom\mat"

In [126]:
gt = np.array([sio.loadmat(os.path.join(gt_dir, i + ".mat"))["inst_map"] for i in names])
k = np.array([sio.loadmat(os.path.join(k_dir, i + ".mat"))["inst_map"] for i in names])
k2 = np.array([sio.loadmat(os.path.join(k2_dir, i + ".mat"))["inst_map"] for i in names])
c = np.array([sio.loadmat(os.path.join(c_dir, i + ".mat"))["inst_map"] for i in names])

gt[gt > 1] = 1
k[k > 1] = 1
k2[k2 > 1] = 1
c[c > 1] = 1

In [127]:
print(dice_loss(torch.from_numpy(gt), torch.from_numpy(gt)))
print(dice_loss(torch.from_numpy(gt), torch.from_numpy(k)))
print(dice_loss(torch.from_numpy(gt), torch.from_numpy(k2)))
print(dice_loss(torch.from_numpy(gt), torch.from_numpy(c)))

tensor(0.)
tensor(0.1562)
tensor(0.1820)
tensor(0.2027)
